In [2]:
from tensorflow import keras
from tensorflow.keras import layers


def build_model(hp):
    units = hp.Int(name='units', min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    optimizer = hp.Choice(name='optimizer', values=['rmsprop', 'adam'])
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [3]:
import keras_tuner as kt


class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        
    def build(self, hp):
        units = hp.Int(name='units', min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation='relu'),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        optimizer = hp.Choice(name='optimizer', values=['rmsprop', 'adam'])
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model
    

hypermodel = SimpleMLP(num_classes=10)

In [4]:
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=2,
    directory='mnist_kt_test',
    overwrite=True,
)
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype('float32') / 255
x_test = x_test.reshape((-1, 28 * 28)).astype('float32') / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 100 Complete [00h 00m 18s]
val_accuracy: 0.975849986076355

Best val_accuracy So Far: 0.9765999913215637
Total elapsed time: 00h 28m 27s


In [7]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [8]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10)]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks
    )
    val_loss_per_epoch = history.history['val_loss']
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f'최상의 에포크 : {best_epoch}')
    return best_epoch

In [9]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(x_train_full, y_train_full, batch_size=128, epochs=int(best_epoch * 1.2))
    return model


best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8206 - loss: 0.6728 - val_accuracy: 0.9318 - val_loss: 0.2404
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.9325 - loss: 0.2389 - val_accuracy: 0.9506 - val_loss: 0.1812
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.9521 - loss: 0.1731 - val_accuracy: 0.9581 - val_loss: 0.1516
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.9598 - loss: 0.1388 - val_accuracy: 0.9632 - val_loss: 0.1303
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.9668 - loss: 0.1171 - val_accuracy: 0.9646 - val_loss: 0.1210
Epoch 6/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step - accuracy: 0.9715 - loss: 0.1010 - val_accuracy: 0.9682 - val_loss: 0.1111
Epoch 7/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.9750 - loss: 0.0888 - val_accuracy: 0.9710 - val_loss: 0.1030
Epoch 8/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.9783 - loss: 0.0

NameError: name 'model' is not defined

In [10]:
from tensorflow import keras

keras.mixed_precision.set_global_policy('mixed_float16')

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

strategy = tf.distribute.TPUStrategy(tpu)
print(f'복제 모델 개수 : {strategy.num_replicas_in_sync}')


def build_model(input_size):
    inputs = keras.Input((input_size, input_size, 3))
    x = keras.applications.resnet.preprocess_input(inputs)
    x = keras.applications.resnet.ResNet50(weights=None, include_top=False, pooling='max')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


with strategy.scope():
    model = build_model(input_size=32)

NameError: name 'tpu' is not defined